# Example notebook for a general script

In [7]:
# process:
# 1. get input file, 
# 2. run the model
# 3. post-processing

In [8]:
# magic lines that avoid re-start 
%load_ext autoreload
%autoreload 2
import csv, random, os
import pybamm as pb;import pandas as pd;import numpy as np;
import os, json,openpyxl,traceback,multiprocessing,scipy.optimize
import matplotlib.pyplot as plt;
import imageio,timeit,random,time, signal
from scipy.io import savemat,loadmat;
from pybamm import constants,exp;import matplotlib as mpl; 
fs=17; 
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
On_HPC = False
if On_HPC:                          # Run on HPC
    Path_NiallDMA = "InputData/" 
    BasicPath=os.getcwd() 
    #Para_file = Path_NiallDMA +  para_csv
else:
    import sys  
    str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
    str_path_1 = os.path.abspath(
        os.path.join(str_path_0,"wip/Rio_Code/Fun_P2_s"))
    sys.path.append(str_path_1) 
    Path_NiallDMA = os.path.expanduser(
        "~/EnvPBGEM_Linux/SimSave/InputData/") # for Linux
    BasicPath =  os.path.expanduser(
        "~/EnvPBGEM_Linux/SimSave/P2_R9_Dim")
    #Para_file = BasicPath+'/Get_Random_sets/'+para_csv

# Get input file or just input

In [100]:
# Functions, will put into the .py file later
def save_rows_to_csv(file_path, rows, header):
    with open(file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)  # Write parameter names as the header row
        writer.writerows(rows)
def generate_combinations(Bounds, Num_tot):
    lower_bounds = []
    upper_bounds = []
    for bound in Bounds:
        lower_bounds.append(bound[0])
        upper_bounds.append(bound[1])
    combinations = []
    for _ in range(Num_tot):
        combination = []
        for lower, upper in zip(lower_bounds, upper_bounds):
            value = random.uniform(lower, upper)
            combination.append(value)
        combinations.append(combination)
    return combinations

def save_combinations_to_csv(combinations, parameter_names, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(parameter_names)  # Write parameter names as the first row
        for combination in combinations:
            writer.writerow(combination)
# Pack input needs to be a list
def Get_Scan_files(
        BasicPath,Target_name,model_options,
        parameter_names,para_short_name,
        Pack,
        rows_per_file,Bundle):
    
    import itertools
    para_dict_Same = {
        "Cycles within RPT":1,
        "RPT temperature":25,
        "Mesh list":[5,5,5,60,20],  
        "Para_Set": "OKane2023",
        "Model option":model_options,
        "Current solvent concentration in the reservoir [mol.m-3]":4541.0,
        "Current electrolyte concentration in the reservoir [mol.m-3]":1000,
        "Ratio of Li-ion concentration change in " 
        "electrolyte consider solvent consumption":1.0,
        'EC initial concentration in electrolyte [mol.m-3]':4541.0,
        'Typical EC concentration in electrolyte [mol.m-3]':4541.0, 
        "Negative electrode number of cracks per unit area [m-2]": 3.18e15,
        "Initial inner SEI thickness [m]": 1.23625e-08,
        "Initial outer SEI thickness [m]": 1.23625e-08,
        "Negative electrode porosity": 0.222393,
        }
    unchange_key2 = list(para_dict_Same.keys())
    unchange_val2 = list(para_dict_Same.values())
    short_pack = [lst for lst in Pack if len(lst) > 1]
    selected_indices = [i for i, lst in enumerate(Pack) if len(lst) > 1]
    shortList_para_short_name = [para_short_name[i] for i in selected_indices]
    shortList_para_short_name.insert(0,"No")
    really_change_val =  [
        list(comb) for comb in itertools.product(*short_pack)]

    change_val =  [
        list(comb) for comb in itertools.product(*Pack)]
    combinations = [[i+1,*elem, *unchange_val2] for i,elem in enumerate(change_val)]
    comb_short   = [[i+1,*elem] for i,elem in enumerate(really_change_val)]
    parameter_names = [*parameter_names,*unchange_key2]
    print("Total cases number is",len(combinations))
    if Bundle:
        # Specify the total number of cases
        total_cases = len(combinations)
        # Specify the number of rows per CSV file, rows_per_file
        # Calculate the number of files needed
        num_files = (total_cases - 1) // rows_per_file + 1
        # Create the target folder
        folder_path = os.path.join(BasicPath, "Get_Random_sets", Target_name)
        os.makedirs(folder_path, exist_ok=True)
        # Write data to each CSV file
        for i in range(num_files):
            file_name = f"Bundle_{i+1}.csv"
            file_path = os.path.join(folder_path, file_name)
            start_row = i * rows_per_file
            end_row = min(start_row + rows_per_file, total_cases)
            rows = combinations[start_row:end_row]
            save_rows_to_csv(file_path, rows, parameter_names)
        filename = BasicPath+f"/Get_Random_sets/{Target_name}/"+f'{Target_name}.csv'
        filename_short = BasicPath+f"/Get_Random_sets/{Target_name}/"+f'{Target_name}_s.csv'
        save_combinations_to_csv(combinations, parameter_names, filename)
        save_combinations_to_csv(comb_short, shortList_para_short_name, filename_short)
        print(f"Combinations saved to '{Target_name}.csv' file.") 
        print(f"CSV files created in folder '{Target_name}'.")
    else:
        filename = BasicPath+"/Get_Random_sets/"+f'{Target_name}.csv'
        save_combinations_to_csv(combinations, parameter_names, filename)
        print(f"Combinations saved to '{Target_name}.csv' file.") 
    return len(combinations)
# pack input can not either be a list or a tuple
def get_list_from_tuple(d, num):
    if d[1] > d[0]:
        if d[1] > 100 * d[0]:
            result_list = (np.exp(np.linspace(np.log(d[0]), np.log(d[1]), num=num))).tolist()
        else:
            result_list = (np.linspace(d[0], d[1], num=num)).tolist()
    else:
        result_list = []
    return result_list
def Get_Scan_Orth_Latin(
        BasicPath,Target_name,model_options,
        parameter_names,para_short_name,
        Pack, num,
        rows_per_file,Bundle):
    
    import itertools; from pyDOE import lhs
    para_dict_Same = {
        "Cycles within RPT":1,
        "RPT temperature":25,
        "Mesh list":[5,5,5,60,20],  
        "Para_Set": "OKane2023",
        "Model option":model_options,
        "Current solvent concentration in the reservoir [mol.m-3]":4541.0,
        "Current electrolyte concentration in the reservoir [mol.m-3]":1000,
        "Ratio of Li-ion concentration change in " 
        "electrolyte consider solvent consumption":1.0,
        'EC initial concentration in electrolyte [mol.m-3]':4541.0,
        'Typical EC concentration in electrolyte [mol.m-3]':4541.0, 
        "Negative electrode number of cracks per unit area [m-2]": 3.18e15,
        "Initial inner SEI thickness [m]": 1.23625e-08,
        "Initial outer SEI thickness [m]": 1.23625e-08,
        "Negative electrode porosity": 0.222393,
        }
    unchange_key2 = list(para_dict_Same.keys())
    unchange_val2 = list(para_dict_Same.values())
    
    Pack_tuple = []; Pack_tuple_index = []
    Pack_list = [];  Pack_list_index  = []
    for i,item in enumerate(Pack):
        if isinstance(item, tuple):
            Pack_tuple.append(item)
            Pack_tuple_index.append(i)
        elif isinstance(item, list):
            Pack_list.append(item)
            Pack_list_index.append(i)
    com_tuple = []; comb_tu_list =[]
    if len(Pack_tuple) > 1:
        for tuple_i in Pack_tuple:
            com_tuple.append( get_list_from_tuple(tuple_i, num) )
        # apply Latin Hypercube:
        #print(com_tuple)
        samples = lhs(len(com_tuple), samples=num)
        for sample in samples:
            combination = []
            for i, candidate_list in enumerate(com_tuple):
                index = int(sample[i] * num)
                combination.append(candidate_list[index])
            comb_tu_list.append(combination)
    else:
        print("error! Pack_tuple must has 2 elements")
    # apply product sampling:
    comb_li_list = [list(comb) for comb in itertools.product(*Pack_list)]
    #print(comb_tu_list)
    #print(comb_li_list)
    Big_Comb = []
    for comb_tu in comb_tu_list:
        for comb_li in comb_li_list:
            big_comb = [0] * (len(comb_tu)+len(comb_li))
            for comb_tu_i,index in zip(comb_tu,Pack_tuple_index):
                big_comb[index] = comb_tu_i
            for comb_li_i,index in zip(comb_li,Pack_list_index):
                big_comb[index] = comb_li_i
            Big_Comb.append(big_comb)
    #print(Big_Comb)
    Big_Comb
    combinations = [[i+1,*elem, *unchange_val2] for i,elem in enumerate(Big_Comb)]

    parameter_names = [*parameter_names,*unchange_key2]
    print("Total cases number is",len(combinations))
    if Bundle:
        # Specify the total number of cases
        total_cases = len(combinations)
        # Specify the number of rows per CSV file, rows_per_file
        # Calculate the number of files needed
        num_files = (total_cases - 1) // rows_per_file + 1
        # Create the target folder
        folder_path = os.path.join(BasicPath, "Get_Random_sets", Target_name)
        os.makedirs(folder_path, exist_ok=True)
        # Write data to each CSV file
        for i in range(num_files):
            file_name = f"Bundle_{i+1}.csv"
            file_path = os.path.join(folder_path, file_name)
            start_row = i * rows_per_file
            end_row = min(start_row + rows_per_file, total_cases)
            rows = combinations[start_row:end_row]
            save_rows_to_csv(file_path, rows, parameter_names)
        filename = BasicPath+f"/Get_Random_sets/{Target_name}/"+f'{Target_name}.csv'
        filename_short = BasicPath+f"/Get_Random_sets/{Target_name}/"+f'{Target_name}_s.csv'
        save_combinations_to_csv(combinations, parameter_names, filename)
        # save_combinations_to_csv(comb_short, shortList_para_short_name, filename_short)
        print(f"Combinations saved to '{Target_name}.csv' file.") 
        print(f"CSV files created in folder '{Target_name}'.")
    else:
        filename = BasicPath+"/Get_Random_sets/"+f'{Target_name}.csv'
        save_combinations_to_csv(combinations, parameter_names, filename)
        print(f"Combinations saved to '{Target_name}.csv' file.") 
    return len(combinations)

def Get_Scan_General(
        BasicPath,Target_name,Para_dict, 
        num,rows_per_file,Bundle=True):
        
    ParaDict_unchange ={}; ParaDict_change_List = {}; ParaDict_change_Tuple = {}
    for key, value in Para_dict.items():
        # depending on what values is:
        # it is a list, then it is changing
        if isinstance(value,list):
            if len(value) >1:
                #print(value)
                ParaDict_change_List[key] = value
            else:
                ParaDict_unchange[key] = value[0]
        # it is a list, so it also changes
        elif isinstance(value,tuple):
            #print(value)
            # create a list based on the tuple:
            ParaDict_change_Tuple[key] = get_list_from_tuple(value, num)
        else:
            ParaDict_unchange[key] = value
    if len(ParaDict_change_List) > 1:
        comb_li_list = [list(comb) for comb in itertools.product(*ParaDict_change_List.values())]
    elif len(ParaDict_change_List) == 0:
        comb_li_list = []
    else:
        comb_li_list = list(ParaDict_change_List.values())
        #print(comb_li_list)
        comb_li_list = comb_li_list [0]
    #comb_li_list
    import itertools; from pyDOE import lhs
    comb_tu_list =[]
    if len(ParaDict_change_Tuple) > 1:
        samples = lhs(len(ParaDict_change_Tuple), samples=num)
        for sample in samples:
            combination = []
            for i, candidate_list in enumerate(ParaDict_change_Tuple.values()):
                index = int(sample[i] * num)
                combination.append(candidate_list[index])
            comb_tu_list.append(combination)
    elif len(ParaDict_change_Tuple) == 0:
        pass
    else:
        comb_tu_list = list(ParaDict_change_Tuple.values())
        comb_tu_list = comb_tu_list[0]
    #comb_tu_list
    Big_Comb = []
    for comb_li in comb_li_list:
        for comb_tu in comb_tu_list:
            if isinstance(comb_tu,list):
                if isinstance(comb_li,list) :
                    Big_Comb.append([*comb_li,*comb_tu,])
                else:
                    Big_Comb.append([comb_li,*comb_tu,]) 
            else:
                if isinstance(comb_li,list):
                    Big_Comb.append([*comb_li,comb_tu,])
                else:
                    Big_Comb.append([comb_li,comb_tu,])  
    if len(Big_Comb) > 1:
        print(f"Total scan number is: {len(Big_Comb)}")
        comb_short = [[i+1,*elem, ] for i,elem in enumerate(Big_Comb)]
        short_name_List =  [
            "Scan No",
            *list(ParaDict_change_List.keys()),
            *list(ParaDict_change_Tuple.keys()),]
        combinations = [[i+1,*elem, *ParaDict_unchange.values()] for i,elem in enumerate(Big_Comb)]
        parameter_names = [
            "Scan No",
            *list(ParaDict_change_List.keys()),
            *list(ParaDict_change_Tuple.keys()),
            *list(ParaDict_unchange.keys())]
    else:
        print(f"Total scan number is: 1")
        combinations = [[1, *ParaDict_unchange.values()]]
        parameter_names = [
            "Scan No",
            *list(ParaDict_unchange.keys())]

    if Bundle:
        # Specify the total number of cases
        total_cases = len(combinations)
        # Specify the number of rows per CSV file, rows_per_file
        # Calculate the number of files needed
        num_files = (total_cases - 1) // rows_per_file + 1
        # Create the target folder
        folder_path = os.path.join(BasicPath, "Get_Random_sets", Target_name)
        os.makedirs(folder_path, exist_ok=True)
        # Write data to each CSV file
        for i in range(num_files):
            file_name = f"Bundle_{i+1}.csv"
            file_path = os.path.join(folder_path, file_name)
            start_row = i * rows_per_file
            end_row = min(start_row + rows_per_file, total_cases)
            rows = combinations[start_row:end_row]
            save_rows_to_csv(file_path, rows, parameter_names)
        filename = BasicPath+f"/Get_Random_sets/{Target_name}/"+f'{Target_name}.csv'
        filename_short = BasicPath+f"/Get_Random_sets/{Target_name}/"+f'{Target_name}_s.csv'
        save_combinations_to_csv(combinations, parameter_names, filename)
        if len(Big_Comb) > 1:
            save_combinations_to_csv(comb_short, short_name_List, filename_short)
        print(f"Combinations saved to '{Target_name}.csv' file.") 
        print(f"CSV files created in folder '{Target_name}'.")
    else:
        filename = BasicPath+"/Get_Random_sets/"+f'{Target_name}.csv'
        filename_short = BasicPath+f"/Get_Random_sets/"+f'{Target_name}_s.csv'
        save_combinations_to_csv(combinations, parameter_names, filename)
        if len(Big_Comb) > 1:
            save_combinations_to_csv(comb_short, short_name_List, filename_short)
        print(f"Combinations saved to '{Target_name}.csv' file.") 
    return parameter_names,combinations


In [ ]:
options_SEI = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",
    
    "lithium plating": "none",
    "lithium plating porosity change":"false",
    "particle mechanics": "constant cracks",
    "loss of active material": "none",

    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
BasicPath =  os.path.expanduser(
   "~/EnvPBGEM_Linux/SimSave/P2_R9_Dim")
Target  = f'/Get_Random_sets/'
if not os.path.exists(BasicPath + Target):
   os.mkdir(BasicPath + Target)

num = 10; rows_per_file = 3
Para_dict = {
    ###################### setting
    # change: 
    #"Scan No",   # by default, 
    "Exp No":[2],   # 2,3,5
    "Ageing temperature":10.0, # 10,25,40

    # unchange
    "Cycles within RPT":1,
    "RPT temperature":25,
    "Mesh list":"[5,5,5,60,20]",  
    "Para_Set": "OKane2023",
    "Model option":options_SEI,
    "Current solvent concentration in the reservoir [mol.m-3]":4541.0,
    "Current electrolyte concentration in the reservoir [mol.m-3]":1000,
    "Ratio of Li-ion concentration change in " 
    "electrolyte consider solvent consumption":1.0,
    'EC initial concentration in electrolyte [mol.m-3]':4541.0,
    'Typical EC concentration in electrolyte [mol.m-3]':4541.0, 

    ###################### parameter that already there
    # change:


    # unchange:
    'Inner SEI lithium interstitial diffusivity [m2.s-1]':[2.68e-18],#1e-19~5e-18
    'Dead lithium decay constant [s-1]': 1e-6,
    'Lithium plating kinetic rate constant [m.s-1]':1e-15,
    'Negative electrode LAM constant proportional term [s-1]':[3e-7,],
    'Positive electrode LAM constant proportional term [s-1]':[3e-7,],
    'Negative electrode cracking rate':[1e-20,],
    'Outer SEI partial molar volume [m3.mol-1]':[4e-5],
    "SEI growth activation energy [J.mol-1]":[1e4,], # 1e4~3.8e4
    "Negative cracking growth activation energy [J.mol-1]":[0.0,],#0
    "Negative electrode diffusivity activation energy [J.mol-1]":[1.7e4,],#1.7e4
    "Positive electrode diffusivity activation energy [J.mol-1]":[1.2e4,],#1.2e4
    "Contact resistance [Ohm]":[0.010,],#0.010
    'Total heat transfer coefficient [W.m-2.K-1]': [20,],#20
    'Initial electrolyte excessive amount ratio':[1.0,], # 1.0 or 0.99
    "Ratio of lithium moles to SEI moles":[2.0,], # 2.0
    "Negative electrode number of cracks per unit area [m-2]": 3.18e15,
    "Initial inner SEI thickness [m]": 1.23625e-08,
    "Initial outer SEI thickness [m]": 1.23625e-08,
    "Negative electrode porosity": 0.222393,
}

Target_name = "Test_2"
parameter_names,combinations = Get_Scan_General(
    BasicPath,Target_name,Para_dict, 
    num,rows_per_file,Bundle=False)

para_one = {}
for para,combination_i in zip(parameter_names,combinations[0]):
   para_one[para]  = combination_i
para_one['Mesh list'] = json.loads(para_one['Mesh list'])
para_one['Mesh list']
para_one